<h1><a href="https://arxiv.org/pdf/1902.08708.pdf">
Distributionally Robust Reinforcement Learning</a></h1>
by Elena Smirnova et al.


<h2>Keynotes</h2>

* The objective of Distributionally Robust Reinforcement Learning (DR-RL) is to maximize performance of the worst-case in an uncertainty set centered at a reference policy.
* Algorithms for DR-RL are proposed
    * The policy improvement benefits from Distribbutionally Robust Optimization (DRO) guarantees.
    * The algorithms are computationally efficient
    * The produced policy is robust in countering with the uncertainty in the test environment.

<h2>Motivations</h2>

* Generalization in reinforcement learning 
    * In previous works, agents achieve excellence at training time but produce sub-optimal behavior in slightly modified test environment
    * Efforts for mitigating the issue include injecting noise at trainig time and detect overfitting through evaluation

* Safe RL in terms of uncertainty in state-transition under worst-case criterion is too pessimistic and conservative
    * Robust MDP framework in model-based setting
    * $\hat{Q}$-learning, robust Q-learning in model-free setting

* Risk-sensitive objective function balances between the risk and utility via weighted sum or exponential utility function.

* Maximum Entropy objectives regularize the entropy of policy state-action distribution thus encouraging exploration

<h2>Backgrounds</h2>

* MDP $M:=(S, A, p, r)$ where $S$ is the state space, $A$ is the action space, $p(s'|s, a)$ is the state-action transition probability and $r\in [r_{min}, r_{max}]$ is a bounded reward function.

* Given a policy $\pi$, the state-action value is $Q^\pi(s,a)=\mathbb{E}_{(s_t, a_t)\sim\pi} [\sum^\infty_{t=0} \gamma^t r(s_t, a_t)|s_0=s, a_0=a]$ where $\gamma$ is the discount faction. The state value is $V^\pi(s)=\mathbb{E}_{a_t\sim\pi}[Q^\pi(s, a_t)]$.

* $P_\pi(s)$ is a state marginal of the trajectory distribution and $d^\pi=(1-\gamma)\sum^\infty_{t=0} \gamma^t P_\pi(s_t=s)$ is the normalized discounted state disctribution. Refer to $D^\pi:=d^\pi\otimes \pi$ is the joint discounted distribution over state-action pairs $(s,a)$ in that $(d^\pi\otimes \pi)(s, a)=d^\pi(s)\pi(a|s)$.

* Standard RL objective is to find a policy $\pi$ with maximal long-term reward 

\begin{align}
J(\pi)&=&\mathbb{E}_{(s,a)\sim\pi} [\sum^\infty_{t=0} \gamma^t r(s,a)]\\
&=&\mathbb{E}_{s\sim d^\pi,a\sim\pi} [Q^\pi(s,a)]
\end{align}

* Given a reference policy $\pi^{ref}$ and corresponding $d^{ref}, D^{ref}$, RL via importance weighting is expressed as follows.
$$J(\pi, D^{ref})=\mathbb{E}_{(s,a)\sim D^{ref}}\big[\frac{d^{\pi}}{d^{ref}(s)}\frac{\pi(a|s)}{\pi^{ref}(a|s)} Q^\pi(s,a)\big]=\mathbb{E}_{(s,a)\sim D^{ref}} [G^{\pi, ref}(s,a)]$$

<h2> Problem Formulation</h2>
 
* During the test, the test sample distribution $D^{test}$ may be different from $D^{ref}$ which is used during training and the objective function of reinforcement learning does not guarantee performance outside $D^{ref}$. 

* Construct an uncertainty set $U_{\epsilon}(D^{ref})$ around $D^{ref}$ to contain the actual test distribution with high confidence. Then the objective of RL beceoms:
$$\pi^{*, test}=argmax_{\pi\in\Pi} min_{D\sim U_\epsilon(D^{ref})} J(\pi, D)$$

* Depending on the sources of uncertainty, consider two setups
    * **`Interpolation`**: when using off-policy approaches to solve RL problems, the policy is updated based on the samples obtained from the policies of previous iterations. The stability of the functiion approximator is to be concerned since the training is based on a finite set of samples.
    * **`Extrapolation`**: when the learning model is deployed, the environmental parameters may be already different from those during traininig.

<h2> Distributionally Robust RL</h2>

<h3> Consider the uncertainty in action distribution</h3>

The set of approximators for $\pi^{ref}$ may be limited. Instead of the uncertainty set of distributions $U_{\epsilon}(D^{ref})$, consider the following uncertainty set of policies centered at $\pi^{ref}$.
$$ U_\epsilon(\pi^{ref})=\{\hat{\pi}:d^{\hat{\pi}}=d^{ref}, D_{KL}(\hat{\pi}(\cdot|s)||\pi^{ref}(\cdot|s))\leq \epsilon_{\pi^{ref}}(s)\ \forall s\in S\}$$
> Note that $\hat{\pi}$ does not put more mass on actions than that of $\pi^{ref}$. Thus if $\pi^{ref}(a|s)=0$, then $\hat{\pi}(a|s)=0$.

Replace $U_{\epsilon}(D^{ref})$ with $U_\epsilon(\pi^{ref})$ in expressing the objective of RL via importance sampling.
$$\pi^{*, test}= argmax_{\pi\in\Pi} min_{\hat{\pi}\sim U_\epsilon(\pi^{ref})} J(\pi, D)=argmax_{\pi\in\Pi} min_{\hat{\pi}\sim U_\epsilon(\pi^{ref})} \mathbb{E}_{(s,a)\sim d^{ref}\otimes \hat{\pi}}[G^{\pi, ref}(s,a)]$$
> Note that $\hat{\pi}$ induces the same $d^{ref}$ as $\pi^{ref}$.

**This objective function aims to find a policy that maximally gains even in the state-action samples that an adversarial policy generates to maximally undermines the performance.**

The optimal adversarial policy can be expressed in closed form by Boltzmann distribution with temperature parameter $\lambda$. It re-weights the reference policy such that the worst-case actions are taken frequently. 

$$\pi^{Adv}_\lambda=\frac{exp(-\frac{1}{\lambda} G^{\pi, ref}(s,a)) \pi^{ref}(a|s)}{\mathbb{E}_{a'\sim\pi^{ref}(\cdot|s)}[exp(-\frac{1}{\lambda} G^{\pi, ref}(s,a')]}$$

Observe that state distribution is not explicitly considered in the objective since the uncertainty is in the action distribution. Use this adversarial policy, the objective of DR-RL is as follows.
$$\pi^{*, test}= argmax_{\pi\in\Pi} \mathbb{E}_{(s,a)\sim d^{ref}\otimes \pi^{Adv}_\lambda}[G^{\pi, ref}(s,a)]$$

And the optimal temperature parameter $\lambda^*(s)$ can be solved via a dual objective of DR-RL.

$$\lambda^*(s):=max_{\lambda>0} log\{\mathbb{E}_{a\sim \pi^{ref}(\cdot|s)} [exp(-\frac{G^{\pi, ref}(s,a)}{\lambda})]\} + \lambda\epsilon$$
> Obviously, when the uncertainty is trivial as $\epsilon\rightarrow 0$, $\lambda\rightarrow \infty$ and the adversarial policy is the same as the reference policy. When the uncertainty is considerable as $\epsilon\rightarrow\infty$, $\lambda\rightarrow 0$ and the adversarial policy takes the worst actions that undermines the long term reward.

**`Generalization guarantees`** can be asymptotically established for uncertainty set with any divergence as the measurement. It is possible to construct a confidence interval around the test performance of the learnt policy when the sample size tends to infinity for KL divergence.


<h3>Consider the uncertainty in future state distribution</h3>

Due to the finite sample size, $D^{ref}$ may not be representative. Construct an uncertainty set of state distributions for the same policy.
$$U_\epsilon(d^{ref})=\{\tilde{d}:\tilde{\pi}=\pi^{ref}, D_{KL} (\tilde{d}||d^{ref})\leq \epsilon_{d^{ref}}\}$$

From this uncertainty set, a new DR-RL objective is obtained.
$$\pi^{*, test}= argmax_{\pi\in\Pi} \mathbb{E}_{(s,a)\sim d_\lambda^{Adv}\otimes \pi^{ref}}[G^{\pi, ref}(s,a)]= argmax_{\pi\in\Pi} \mathbb{E}_{s\sim d_\lambda^{Adv}}[G^{\pi, ref}(s)]$$
$$d_\lambda^{Adv}(s)=\frac{exp(-\frac{1}{\lambda} G^{\pi, ref} (s)) d^{ref}(s)}{\mathbb{E}_{s'\sim d^{ref}}[exp(-\frac{1}{\lambda} G^{\pi, ref}(s'))]}$$

> Note that since all $\pi\in U_\epsilon(d^{ref})$ choose the same actions as $\pi^{ref}$, action distribution is no longer explicitly considered in the objective anymore. The optimal adversarial state distribution $d^{Adv}_\lambda$ is the worst case state-distribution induced in a similar way to Boltzmann policy. 


Depending on $\epsilon$, $d^{Adv}_\lambda$ encourages visiting the low reward states. From the following observation, we shall see that $d^{Adv}_\lambda$ changes the state-action visiting rate by re-weighting the state performance gain.
$$d^{Adv}_\lambda(s)\pi^{ref}(a|s)\propto d^{ref}(s)\big(\pi^{ref}(a|s)exp(-\frac{G^{\pi, ref}(s)}{\lambda})\big)$$

<h3>General DR-RL objective</h3>

Consider a unified uncertainty set $U_\epsilon=\{d^\pi\otimes \pi: D_{KL}(d^\pi\otimes\pi||d^{ref}\otimes\pi^{ref})\leq \epsilon_{\pi^{ref}}(s) + \epsilon_{d^{ref}}\ \forall(s,a)\in S\times A\}$

Using this uncertainty set, the general DR-RL objective is as follows.
$$\pi^{*,test}=argmax_{\pi\in \Pi}\mathbb{E}_{(s,a)\sim d^A_\lambda\otimes \pi^A_\lambda}[G^{\pi, ref}(s,a)]$$

In the case of convex and bounded loss function, a Distributionally Robust Optimization problem with a uncertainty set can be formulated into a convex surrogate to variance-regularized objective. Likewise, the DR-RL objective can be reformulated into a regularized objective without deriving the closed forms $d^A_\lambda\otimes, \pi^A_\lambda$ beforehand.
$$\pi^{*,test}=argmax_{\pi\in \Pi}\mathbb{E}_{(s,a)\sim d^{ref} \pi^{ref}}[G^{\pi, ref}(s,a)]-\sqrt{2\epsilon} \sqrt{Var_{(s,a)\sim d^{ref} \pi^{ref}}[G^{\pi, ref}(s,a)]}$$

> While the the regularization in DR-RL penalizes the variance of performance metric, the KL-divergence regularized Maximal Entropy RL as in Todorov's paper encourages exploration to recover multiple modes of near-optimal behavior. 
$$J(\pi)=\mathbb{E}_{(s,a)\sim\pi}[Q^\pi_{soft}(s,a)-\alpha D_{KL}(\pi(\cdot|s)||\pi^{ref}(\cdot|s))]$$
$$Q^\pi_{soft}(s,a)=\mathbb{E}_{(s_t, a_t)\sim\pi}[\sum^\infty_{t=0}\gamma^t (r(s_t, a_t)-\alpha D_{KL}(\pi(\cdot|s_t)||\pi^{ref}(\cdot|s_t)))|s_0=s, a_0=a]$$ And the regularized MaxEnt policy has the closed form as a Boltzmann distribution.
$$\pi^B_\alpha(a|s)=\frac{exp(\frac{1}{\alpha}Q^{\pi^B}_{soft}(s,a))\pi^{ref}(a|s)}{\mathbb{E}_{a'\sim\pi^{ref}}[exp(\frac{1}{\alpha}Q^{\pi^B}_{soft}(s,a')]}$$


<h2> Algorithms</h2>

<h3> Distributionally Robust Policy Iteration</h3>

The algorithm generates an improving sequence of policies and the algorithm stabilizes the learning. It is both applicable to on policy and off policy setups.

**Algorithm**
<ol>Given initial policy $\pi$ and confidence interval $\delta$. Repeat the following steps.
    <li>D: Produce n samples from policy $\pi$ </li>
    <li>Compute performance metric $G^{\pi}(s,a)$ on D</li>
    <li>Determine $\epsilon\leftarrow \frac{log{\frac{1}{\delta}}}{n} and solve \lambda\leftarrow \lambda^*$</li>
    <li>D': Reweight samples in D using the adversarial state-action distribution $d^A_\lambda\otimes \pi^A_\lambda$.</li>
    <li>$\pi$: improve and update the policy using sample set $D'$
    </ol>
    
<h3> Distributionally Robust Soft Q-learning</h3>

Write down the optimal adversarial policy for the KL-divergence regularized Maximal Entropy RL. 

$$\pi^{Adv, MaxEnt}_\alpha(a|s)=\frac{exp((\frac{1}{\alpha}-\frac{1}{\lambda})Q^{\pi^B}_{soft}(s,a))\pi^{ref}(a|s)}{\mathbb{E}_{a'\sim\pi^{ref}}[exp((\frac{1}{\alpha}-\frac{1}{\lambda})Q^{\pi^B}_{soft}(s,a')]}$$

> Observe that the policy appears as a change of sampling temperature.

Meanwhile, the dual problem that solves $\lambda$ can be written in the following way.
$$\lambda^{*, MaxEnt}(s):=max_{\lambda>0} log\sum_{a\in A}[(\frac{1}{\alpha}-\frac{1}{\lambda})exp(Q^\pi_{soft}(s,a))] + \lambda\epsilon$$

**Algorithm**
<ol> A confidence interval $\delta$ as well as the temperature $\alpha$ are given. Initialize with a random $Q_{soft}(s,a)$ and a random state $s$. Repeat the following steps for $N$ steps.
    <li>$\epsilon\leftarrow log(\frac{1}{\delta})/n$</li>
    <li>$\lambda\leftarrow$ 1-d optimization w.r.t $\lambda^{*, MaxEnt}$.</li>
    <li>Sample action in current state using adversarial Boltzmann policy $\pi^{Adv, MaxEnt}_\alpha$</li>
    <li>Update $Q_{soft}(s,a)$ only for current state</li>
    <li>Move to next state following the system dynamics.</li>
    </ol>

<h1>Evaluation</h1>

Consider two problem setups. One is interpolation setup in which DR-RL is used to stabilize the learning process. The other one is extrapolation setup where DR-RL encounters distributional shift between training and test environment, such as srecommender system.

* The evaluation environment is a grid-world. The transitioning is deterministic during training yet stochastic during testing.
* Three algorithms are tested: Q-learning, Soft Q-learning and DR Soft Q-learning. The used neural networks have only one hidden layer.
* Compared with Soft Q-learning, DR Soft Q-learning converges to its own optimal Q-value faster.
* In stochastic testing environment, DR Soft Q-learning has higher average return compared with other two when the confidence level $\delta$ is high, hence low uncertainty bound $\epsilon$. When the confidence level decreases, the average return increases. Also, Q-learning and Soft Q-learning exhibits more varied performance (confidence interval) than DR Soft Q-learning.


<h1>Comments</h1>

* Distributionally Robust Optimization can be used to include more informative distributions that “drive out” the corrupted samples instead of inequally weighting all the samples as in the empirical distribution. Intuitively This can benefit reinforcement learning. Yet this seems to be the first paper on combining the two concepts.
* In this paper, a policy is solved in an adversarial manner to generate samples that degrade the performance overall. Yet the finally induced policy is robust against the uncertainty in the environment.
* This paper reminds me of our Trojan-attack project. In the project, we introduce sparse and poisoned samples during the training time and expect the leanring model to be influenced by the samples. In DR-RL, the adversarial policy puts more weights on samples that are less seen and likely to undermine the performance thus the learnt policy can adapt to the samples. It is interesting to think about whether this DR-RL can make a model more defensive against or vulnerable to those attacks.